In [0]:
from pyspark.sql.types import *
customer_schema_trigger= StructType(
    fields = [
        StructField('customer_id',IntegerType()),
        StructField('customer_name',StringType()),
        StructField('date_of_birth',DateType()),
        StructField('telephone',StringType()),
        StructField('email',StringType()),
        StructField('member_since',DateType()),
        StructField('created_timestamp',TimestampType()),

    ]
)

In [0]:
customer_trigger_df = (spark.readStream.format('json')
              .schema(customer_schema_trigger)
              .load("/Volumes/gizmobox/landing/operational_data/customers_stream/")

)

In [0]:
from pyspark.sql.functions import col, current_timestamp, row_number, split
customer_trigger_transform_df = (
    customer_trigger_df
    .withColumn("fname", split(col("customer_name"), " ").getItem(0))
    .withColumn("lname", split(col("customer_name"), " ").getItem(1))
    .withColumn("created_time", current_timestamp())
    .filter(col("customer_id").isNotNull())
)



In [0]:
stream_trigger_query = (customer_trigger_transform_df.writeStream
                .format("delta")
                .trigger(availableNow=True)
                .option("checkpointLocation","/Volumes/gizmobox/landing/operational_data/customers_stream/checkpoint_trigger")
                .toTable("gizmobox.bronze.customer_trigger")
                
                
                )